# TreeIndex 效果测试

结论：

- `ALL_LEAF`, 又慢总结的也未必详细
- 其他3种查询，`ROOT`, `SELECT_LEAF`, `SELECT_LEAF_EMBEDDING` 查询时间变化不大（1个和10个的比较）
- `ROOT` 最快，然后是 `SELECT_LEAF_EMBEDDING`，`SELECT_LEAF` 比 `SELECT_LEAF_EMBEDDING` 稍慢
- 创建索引是线性的，文档越多，越慢

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    # model="qwen2", 
    model="glm4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 3.3 s, sys: 448 ms, total: 3.75 s
Wall time: 3.37 s


## 加载数据

In [1]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "颐和园",
    "北海公园",
    "故宫",
    "八达岭长城",
    "天坛公园",
    "恭王府",
    "圆明园",
    "景山公园",
    "中山公园",
    "陶然亭公园",
]

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

ValidationError: 1 validation error for Document
text
  none is not an allowed value (type=type_error.none.not_allowed)

## 创建索引

根据创建文档个数的测试：

- 1个文档，31.1s，37个node
- 10个文档，2min 58s，225个node

In [3]:
%%time

from llama_index.core import TreeIndex
from llama_index.core.node_parser import SentenceSplitter

index = TreeIndex.from_documents(
    documents,
    # num_children=5,
    # transformations=[SentenceSplitter(chunk_size=128,chunk_overlap=10)]
)

CPU times: user 907 ms, sys: 21.1 ms, total: 928 ms
Wall time: 2min 58s


In [4]:
len(index.docstore.docs.items())

225

In [5]:
for k,v in index.docstore.docs.items():
    node = v
    document_id = k

    children_doc_ids = index.index_struct.get_children(node)

    if len(children_doc_ids) == 0:
        continue

    print(f"parent_doc_id: {document_id}")


    for node_num, children_doc_id in children_doc_ids.items():

        children_node = index.docstore.get_node(children_doc_id)

        print("\t" + f"child_doc_id: {children_doc_id} ")

        # file_name = children_node.metadata.get("file_name")

        # print("\t" + f"child_doc_id: {children_doc_id} (source: [{file_name}])")

parent_doc_id: a665b511-278e-40d2-82b9-9f0325dd40a0
	child_doc_id: 970dc1d8-d362-4b25-ba27-49466c40e002 
	child_doc_id: c6c11cc4-ab4f-453a-a766-3dd31b27fba6 
	child_doc_id: 6acbbf39-269d-459e-9cac-83a8b0ff03c4 
	child_doc_id: 49bf0ce6-db4f-4217-b049-78eb668156fc 
	child_doc_id: c17911f8-4bdd-4761-bc00-a83a83229213 
	child_doc_id: e802a145-2654-4aff-9315-71801c6e7d6b 
	child_doc_id: 26222c76-bb84-406a-a9b4-a0209bd54879 
	child_doc_id: eb64d07e-1f6e-43ca-ac2c-ae61e4006f22 
	child_doc_id: a62d5728-8c58-4e74-a4d6-1bf65454fc80 
	child_doc_id: 8067c02d-3726-4e1a-a448-0948c1840578 
parent_doc_id: 295e4a1c-d9b4-4268-ad16-d31e2e8bca08
	child_doc_id: b1d65dda-527d-4a3b-8035-1eae258b540e 
	child_doc_id: eb6b866c-3529-4017-96a5-e91f0d60fb07 
	child_doc_id: d0e52df8-616b-436b-8043-a67532ac0f3e 
	child_doc_id: f6f45909-9233-4147-a6f6-64438bf1fe90 
	child_doc_id: d9cd4532-4f28-4bf3-a7a8-7f36cb3caa0d 
	child_doc_id: a6544b08-625b-4d50-84d0-9eea5b7b4d2b 
	child_doc_id: 467ada78-f47a-4ab0-ac90-513b0d75b

In [6]:
for k,v in index.docstore.docs.items():
    node = v
    document_id = k

    children_doc_ids = index.index_struct.get_children(node)

    if len(children_doc_ids) == 0:
        continue

    print(f"parent_doc_id: {document_id}")

    print(f"\ttext: {node.text}")

parent_doc_id: a665b511-278e-40d2-82b9-9f0325dd40a0
	text: 颐和园是中国清朝时期的皇家园林，位于北京市西郊海淀区新建宫门路19号。始建于1750年，原名清漪园，后因火灾被毁，于1886年重建并更名为颐和园。该园以昆明湖、万寿山为基础，借鉴了杭州西湖的风景设计，成为保存最完整的皇家园林之一。

1961年，颐和园被列为全国重点文物保护单位，1998年被联合国教科文组织列入《世界遗产名录》。园内主体建筑为大报恩延寿寺，还有一条长达700多米的长廊。1900年八国联军侵占北京时，颐和园遭到破坏，后于1902年修复。

颐和园占地面积约3平方公里，气候为温带季风气候。民国时期，颐和园开始向个人提供租住服务，并增设娱乐设施。1950年开始对颐和园进行详细规划，恢复历史风貌。在“文化大革命”期间，颐和园受到一定程度的破坏，但许多重要建筑得以保留。

总体来说，颐和园是中国古代园林艺术的杰出代表，具有重要的历史、文化和艺术价值。
parent_doc_id: 295e4a1c-d9b4-4268-ad16-d31e2e8bca08
	text: 颐和园自1984年起进行了多次修缮与开放。1984年整修德和园，陈设慈禧和光绪蜡像及文物，同年9月10日开放。1986年开始修复苏州街，1990年9月15日正式开放。2010年至2012年对四大部洲建筑群进行大规模修缮，恢复历史风貌。2011年2月10日，颐和园霁清轩南侧老园墙坍塌，次年2月完成抢修。2014年启动建国后最大修缮工程——园墙修缮。颐和园模仿江南园林，如昆明湖景观设计仿杭州西湖。建筑形式多样，融合中国绘画、诗歌意境，展现皇家园林宏大气势与色彩协调。园内重要景点包括苏州街、宝云阁、排云殿、长廊、谐趣园等。颐和园藏文物反映近代中国兴衰历程。
parent_doc_id: d680c8d6-6a67-48be-a47f-b9c881344da8
	text: Summary:

In 1902 (Guangxu 28th year), Empress Ci Xi returned to Beijing and, despite the country's precarious situation, she restored and greatly enriched the 

## 查询

### SELECT_LEAF

丢失部分信息： 1900年八国联军攻入北京 ...

丢失信息的原因可能是因为 `child_branch_factor`，默认是 `1`

和 SummaryIndex（3min 5s） 比速度满意

多文档测试：

- 1个文档，19.3 s
- 10个文档，18.6 s

In [7]:
%%time

from llama_index.core.indices.tree.base import TreeRetrieverMode

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.SELECT_LEAF,
    child_branch_factor=3,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园是在清漪园的基础上建造而成的。最初，清漪园是乾隆皇帝为庆祝母亲六十寿辰而建的一座园林。随着清朝的盛世到来，圆明园在乾隆年间得到了大规模扩建，成为当时中国最宏伟的皇家园林之一。清漪园作为圆明园的一部分，也在此期间逐渐发展壮大。

然而，1860年英法联军焚毁了圆明园，包括清漪园在内的许多建筑和景观被毁。之后，慈禧太后在光绪二十年（1894年）决定重建清漪园，并将其改名为颐和园。颐和园的重建工程历时十年，于1902年竣工。

颐和园不仅是中国园林艺术的杰作，也是融合了中国传统建筑艺术与西洋建筑手法的典范。它以昆明湖为中心，万寿山为背景，拥有长廊、佛香阁、十七孔桥等著名景点。在慈禧太后的统治时期，颐和园成为其政治和生活的中心。

颐和园不仅具有极高的历史价值和文化意义，还承载着中华民族近代屈辱的历史记忆。它不仅是中华文明辉煌的象征，也是全人类深刻检讨和反思的文化遗存。如今，颐和园已成为北京著名的旅游景点，每年吸引大量国内外游客参观游览。
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 18.6 s


### ROOT

速度更快，内容上没有丢失主要部分，基本满意

多文档测试：

- 1个文档，6.95 s
- 10个文档，5.25 s

In [8]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.ROOT,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园，位于北京，是一座具有深厚历史和文化价值的著名园林。它原为清朝皇家园林，始建于18世纪末，是乾隆皇帝为其母亲祝寿而建。颐和园融合了山水园林艺术与建筑美学，其设计理念体现了中国古代的哲学思想。

在19世纪中叶，由于战乱和政治动荡，颐和园遭受严重破坏。但在20世纪初，经过多次修复，颐和园逐渐恢复了昔日的风采。它不仅是中国园林艺术的杰作，也是世界文化遗产之一。

颐和园的主要建筑包括长廊、佛香阁、十七孔桥等，这些均展示了高超的建筑技艺和精美的装饰艺术。园内还有昆明湖、万寿山等自然景观，与人工建筑相得益彰。

在文化大革命期间，颐和园再次遭受破坏，但随后得到了修复和保护。如今，颐和园不仅是国内外游客的热门旅游目的地，也是举办各类文化活动的重要场所。
CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 5.25 s


### ALL_LEAF

速度最慢的一个，甚至慢于 SummaryIndex (3min 5s) -- 可能是文档多了

当文档多了以后，线性的变慢。

多文档测试：

- 1个文档，3min 2s
- 10个文档，15min 2s

In [9]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.ALL_LEAF,
    child_branch_factor=3,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园，原名清漪园，始建于1750年，是中国历史悠久的皇家园林之一。这座园林历经多次变迁，包括扩建、重建以及战争破坏等。在民国时期，颐和园经历了多项改造，丰富了公园内的娱乐设施。1937年“七七事变”后，颐和园一度停止开放四月余。经过多次修缮和改造，颐和园逐渐恢复了昔日的辉煌。如今，它已成为中国著名的旅游景点，以其精美的园林设计、丰富的历史文化内涵以及独特的建筑风格吸引着众多国内外游客前来参观游览。颐和园不仅是皇家园林的代表，也是中国园林艺术的典范之一。
CPU times: user 1.2 s, sys: 21.3 ms, total: 1.22 s
Wall time: 15min 2s


### SELECT_LEAF_EMBEDDING

速度没问题，信息有丢失。

多文档测试：

- 1个文档，15.8 s
- 10个文档，16.4 s

In [10]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.SELECT_LEAF_EMBEDDING,
    child_branch_factor=3,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园的历史悠久且丰富。最初名为清漪园，始建于乾隆十五年（1750年），是清朝皇家园林之一。在清王朝国势鼎盛时期，乾隆皇帝对文玩鉴赏有独到之处，大量搜罗古代铜器、瓷玉及书画等珍品，使得清漪园陈设丰富，当时著录的陈设多达四万余件。

然而，鸦片战争后，清朝国力转衰，清漪园陈设有所裁撤。咸丰五年（1855年），清漪园陈设减少至三万七千五百八十三件。1860年，包括清漪园在内的北京西北郊的五座大型皇家园林遭到英法联军焚掠，大量文物被毁。光绪十二年（1886年），清廷开始重建清漪园，两年后改名颐和园，成为慈禧太后归政后的休养之地。

在光绪年间，颐和园得到了大力修复和充实，八方珍宝源源不断涌入。文昌院、德和园等景点相继建成，苏州街也进行了修复。1984年，对园内的德和园进行整修油饰，并陈设了慈禧和光绪的蜡像及有关文物。1990年，苏州街正式开放。

颐和园在保护和发展方面也做出了努力。1989年编制了第一部《颐和园总体保护规划》，随后每隔五年细化完善规划项目。2006年，佛香阁、排云殿、长廊三大标志性建筑修缮工程竣工。2010年至2012年，四大部洲建筑群进行了大规模修缮。

总之，颐和园历经多次兴衰，见证了清朝的辉煌与衰落，如今已成为北京园林系统和国保单位中编制规划较早的公园之一。
CPU times: user 304 ms, sys: 1.48 ms, total: 305 ms
Wall time: 16.4 s


In [11]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.SELECT_LEAF_EMBEDDING,
    child_branch_factor=5,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园位于北京市海淀区，是清朝皇家园林之一。以下是颐和园的历史概述：

1. **起源与发展**：颐和园始建于1750年，最初名为清漪园，由乾隆帝为庆祝母亲六十寿辰而建。1860年英法联军入侵北京时，清漪园被焚毁。1888年，慈禧太后重建园林，并更名为颐和园。

2. **建筑与景观**：颐和园占地约290公顷，以昆明湖、万寿山为核心，拥有众多亭台楼阁、桥梁水榭等建筑。其中，长廊是世界上最长的画廊之一，佛香阁、排云殿等建筑具有很高的艺术价值。

3. **历史事件**：
   - 1984年：对德和园（大戏台）进行整修油饰，陈设了慈禧和光绪的蜡像及有关文物。
   - 1986年：修复苏州街（买卖街），包括房屋、牌楼等建筑。
   - 1990年：昆明湖清淤工程完成，湖水重新放水。
   - 2000年：编制《2000-2020年颐和园公园发展保护规划》，并逐步实施部分项目。
   - 2006年：佛香阁、排云殿、长廊三大标志性建筑修缮工程竣工。
   - 2010年至2012年：四大部洲建筑群进行大规模修缮。

4. **文化价值**：颐和园不仅具有极高的艺术价值，还承载着丰富的历史文化。它见证了清朝的兴衰，是中国园林艺术的杰出代表。

5. **现状与保护**：如今，颐和园已成为北京园林系统和国保单位中编制规划较早的公园之一。在保护和修复方面，我国政府投入了大量资金和人力，以确保这一世界文化遗产得以传承下去。

总之，颐和园是一座融合了自然景观、人文历史和艺术价值的皇家园林，是中国乃至世界的宝贵财富。
CPU times: user 460 ms, sys: 5.28 ms, total: 465 ms
Wall time: 28.9 s


## 结论

目前需求，比较适合使用 `TreeRetrieverMode.ROOT` 的 `TreeIndex` 替代 `SummaryIndex`